### Decorators 2

How to parameterize decorators

In [39]:
# my imports
import wat

In [1]:
# decorator
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        time_elapsed = perf_counter() - start
        print(f'Run time: {time_elapsed:.6f}s / {time_elapsed*1000:.6f}ms')
        return result

    return inner

In [2]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

def fib(n):
    return calc_fib_recurse(n)

In [3]:
fib(20)

6765

In [4]:
# decorate fib with timed decorator using long syntax
fib = timed(fib)

In [6]:
fib(30)

Run time: 0.084710s / 84.710193ms


832040

In [7]:
# decorator
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            time_elapsed = end - start
            total_elapsed += time_elapsed
        avg_elapsed = total_elapsed / 10
        print(f'Avg run time: {avg_elapsed:.6f}s / {avg_elapsed*1000:.6f}ms')
        return result
    return inner 

In [8]:
def fib(n):
    return calc_fib_recurse(n)

In [9]:
# decorate fib with timed decorator using long syntax
fib = timed(fib)

In [10]:
fib(28)

Avg run time: 0.027735s / 27.735200ms


317811

Iterator value of 10 has been hardcoded. Make it a parameter.

In [12]:
# decorator
def timed(fn, num_reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            time_elapsed = perf_counter() - start
            total_elapsed += time_elapsed
        avg_elapsed = total_elapsed / num_reps
        print(f'Avg run time over {num_reps} iterations: {avg_elapsed:.6f}s / {avg_elapsed*1000:.6f}ms')
        return result

    return inner 

Decorate fib function using long syntax @syntax will not work

In [13]:
def fib(n):
    return calc_fib_recurse(n)

In [14]:
fib = timed(fib, 5)

In [15]:
fib(28)

Avg run time over 5 iterations: 0.032290s / 32.290467ms


317811

In [16]:
@timed(5)
def fib(n):
    return calc_fib_recurse(n)

TypeError: timed() missing 1 required positional argument: 'num_reps'

Let's explore

In [28]:
def dec(fn):
    print('running dec')

    def inner(*args, **kwargs):
        print('running inner')
        return fn(*args, **kwargs)
    
    return inner
    

In [29]:
@dec
def my_func():
    print('running my_func')

running dec


note running dec was printed above.
`@dec` decorator is a call to the `dec` decorator

this is the same as the below hard syntax below

In [30]:
def my_func():
    print('running my_func')

my_func = dec(my_func)

running dec


In [31]:
my_func()

running inner
running my_func


What if `dec` was not the decorator, but instead created and returned a decorator

In [37]:
def dec_factory():
    print('running dec_factory')
    
    def dec(fn):
        print('running dec')
        
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
            
        return inner
        
    return dec

In [38]:
# the decorator is being returned by the factory function
dec = dec_factory()

running dec_factory


In [43]:
wat.code / dec

value: <function dec_factory.<locals>.dec at 0x7f737d3a2480>
type: function
signature: def dec(fn)
source code:
    def dec(fn):
        print('running dec')
        
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
            
        return inner



In [44]:
def my_func():
    print('running my_func')

In [45]:
my_func = dec(my_func)

running dec


In [46]:
wat.code / my_func

value: <function dec_factory.<locals>.dec.<locals>.inner at 0x7f737d3eb6a0>
type: function
signature: def inner(*args, **kwargs)
source code:
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)



In [47]:
my_func()

running inner
running my_func


In [50]:
@dec
def my_func():
    print('running my_func')

running dec


In [51]:
my_func()

running inner
running my_func


In [52]:
@dec_factory()
def my_func():
    print('running my_func')

running dec_factory
running dec


In [53]:
my_func()

running inner
running my_func


In [54]:
def my_func():
    print('running my_func')

my_func = dec_factory()(my_func)

running dec_factory
running dec


In [55]:
my_func()

running inner
running my_func


Now inject some parameters

In [56]:
def dec_factory(a, b):
    print('running dec_factory')
    
    def dec(fn):
        print('running dec')
        
        def inner(*args, **kwargs):
            print('running inner')
            print(f'a={a}, b={b}')
            return fn(*args, **kwargs)
            
        return inner
        
    return dec

In [57]:
dec = dec_factory(10, 20)

running dec_factory


In [59]:
@dec
def my_func():
    print('running my_func')

running dec


In [60]:
my_func()

running inner
a=10, b=20
running my_func


In [61]:
@dec_factory(100, 200)
def my_func():
    print('running my_func')

running dec_factory
running dec


In [62]:
my_func()

running inner
a=100, b=200
running my_func


In [63]:
def my_func():
    print('running my_func')

my_func = dec_factory(150, 250)(my_func)

running dec_factory
running dec


In [64]:
my_func()

running inner
a=150, b=250
running my_func


In [79]:
# my exploring
for a in range(2):
    for b in range(2):
        print(a,b)
        
        def my_func():
            print('running my_func')
            
        my_func = dec_factory(a,b)(my_func)
        print(my_func())
        print()

0 0
running dec_factory
running dec
running inner
a=0, b=0
running my_func
None

0 1
running dec_factory
running dec
running inner
a=0, b=1
running my_func
None

1 0
running dec_factory
running dec
running inner
a=1, b=0
running my_func
None

1 1
running dec_factory
running dec
running inner
a=1, b=1
running my_func
None



Make timed decorator a decorator factory

In [80]:
# original decorator
def timed(fn, reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            time_elapsed =  end - start
            total_elapsed += time_elapsed
        avg_elapsed = total_elapsed / reps
        print(f'Avg run time over {reps} iterations: {avg_elapsed:.6f}s / {avg_elapsed*1000:.6f}ms')
        return result

    return inner 

In [81]:
def dec_factory(reps):
    def timed(fn):
        from time import perf_counter
    
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                time_elapsed =  end - start
                total_elapsed += time_elapsed
                
            avg_elapsed = total_elapsed / reps
            print(f'Avg run time over {reps} iterations: {avg_elapsed:.6f}s / {avg_elapsed*1000:.6f}ms')
            return result
        return inner
    return timed

In [84]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

In [89]:
@dec_factory(10)
def fib(n):
    return calc_fib_recurse(n)

In [90]:
fib(28)

Avg run time over 10 iterations: 0.025808s / 25.808314ms


317811

Clean up decoratory factory

In [96]:
def timed(reps):
    def dec(fn):
        from time import perf_counter
    
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                time_elapsed =  end - start
                total_elapsed += time_elapsed
                
            avg_elapsed = total_elapsed / reps
            print(f'Avg run time over {reps} reps: {avg_elapsed:.6f}s / {avg_elapsed*1000:.6f}ms')
            return result
        return inner
    return dec

In [97]:
@timed(reps=15)
def fib(n):
    return calc_fib_recurse(n)

In [98]:
fib(28)

Avg run time over 15 reps: 0.028781s / 28.780974ms


317811

Add additional functionaliy

In [171]:
from functools import wraps
def timed(reps=1):
    def decorator(fn):
        from time import perf_counter

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            elapsed_times = []
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                elapsed_time =  end - start
                elapsed_times.append(elapsed_time)
                
            min_elapsed_time = min(elapsed_times)
            avg_elapsed_time = sum(elapsed_times) / len(elapsed_times)
            max_elapsed_time = max(elapsed_times)
            
            print(f'Number iterations {reps}')
            print(f'Elapsed times:')
            print(f'  min: \t{min_elapsed_time:.6f}s / {min_elapsed_time*1000:.6f}ms')
            print(f'  avg: \t{avg_elapsed_time:.6f}s / {avg_elapsed_time*1000:.6f}ms')
            print(f'  max: \t{max_elapsed_time:.6f}s / {max_elapsed_time*1000:.6f}ms')
            enum_et = [et for et in enumerate(elapsed_times)]
            sorted_et = sorted(enum_et, key=lambda et: et[1])
            for i, et in sorted_et:
                print(f'  {i}: \t{et:.6f}s / {et*1000:.6f}ms')
            return result
        return inner
    return decorator

In [172]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

In [173]:
@timed(reps=10)
def fib(n):
    return calc_fib_recurse(n)

In [174]:
fib(28)

Number iterations 10
Elapsed times:
  min: 	0.025608s / 25.607520ms
  avg: 	0.031918s / 31.917897ms
  max: 	0.039613s / 39.612780ms
  7: 	0.025608s / 25.607520ms
  4: 	0.026856s / 26.856450ms
  9: 	0.027134s / 27.134460ms
  5: 	0.029239s / 29.238750ms
  6: 	0.029487s / 29.486790ms
  0: 	0.032585s / 32.585490ms
  8: 	0.035451s / 35.451180ms
  3: 	0.035478s / 35.478270ms
  2: 	0.037727s / 37.727280ms
  1: 	0.039613s / 39.612780ms


317811

In [175]:
wat.code / fib

value: <function fib at 0x7f737cf1dee0>
type: function
signature: def fib(n)
source code:
@timed(reps=10)
def fib(n):
    return calc_fib_recurse(n)



Extra:
Decorator factory template

In [ ]:
from functools import wraps

# Decorator with parameters
def decorator_factory(arg):
  def decorator(func):
    @wraps(func) # For preserving the metadata of func.
    def wrapper(*args,**kwargs):
      # Do stuff before func possibly using arg...
      result = func(*args,**kwargs)
      # Do stuff after func possibly using arg...
      return result
    return wrapper
  return decorator

# Decorator function
@decoratr_factory(arg='x')
def foo(bar):
    return bar

# Execute function
foo()

    